In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict

/var/folders/zj/v2lfvlss2_764d771fy6f6yc0000gn/T/ipykernel_16516/1767560608.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
songs = pd.read_csv('songdata (1).csv')

In [3]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
songs.isna().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [5]:
songs['text'] = songs['text'].replace(r'\n','')


In [6]:
str=''
for i in range(0,len(songs)):
    str=str+songs['artist'][i]+" "

In [7]:
artists=str.split()

In [8]:
unique_artists=set(artists)

In [9]:
len(unique_artists)

989

In [10]:
# this class is being used to save the recommendations
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def message(self, song, recom_song):
        rec_num = len(recom_song)
        rec=[]
        for i in range(rec_num):
            x={"Song":recom_song[i][1],"Artist":recom_song[i][2]}
            rec.append(x)
        return rec
        
    def recommend(self, recommendation):
        song = recommendation['song']
        number_songs = recommendation['number_songs']
        recom_song = self.matrix_similar[song][:number_songs]
        recommen_list=self.message(song=song, recom_song=recom_song)
        return recommen_list

In [11]:
# class CollaborativeFilteringRecommender:

In [12]:
def get_selection(artist,song,text,rec_num):
    global songs
    x={'artist':artist,'song':song,'text':text}
    x=pd.DataFrame([x])
    songs = songs.sample(n=3000).drop('link', axis=1).reset_index(drop=True)
    songs=pd.concat([songs,x])
    tfidf = TfidfVectorizer(analyzer='word', stop_words='english')
    lyrics_matrix = tfidf.fit_transform(songs['text'])
    cosine_similarities = cosine_similarity(lyrics_matrix) 
    similarities = {}
    for i in range(len(cosine_similarities)):
        similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
        similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'].iloc[x], songs['artist'].iloc[x]) for x in similar_indices][1:]
    recommedations = ContentBasedRecommender(similarities)
    rec_data = {
    "song": songs['song'].iloc[3000],
    "number_songs": rec_num }
    rec_songs=recommedations.recommend(rec_data)
    return rec_songs

The 5 recommended songs for Thumbs are:


[{'Song': 'Boathouse', 'artist': 'Gordon Lightfoot'}, {'Song': 'Living For The City', 'artist': 'Ray Charles'}, {'Song': '7 Minutes In Heaven', 'artist': 'Fall Out Boy'}, {'Song': 'Papa De Da Da', 'artist': 'Louis Armstrong'}, {'Song': 'Hey Jude', 'artist': 'Paul McCartney'}]
